In [1]:
import cv2 as cv       #v3.4.2
import numpy as np     #v1.18.1
import pandas as pd    #v1.0.3
import scipy.spatial   #v1.4.1
import sklearn.cluster #v0.22.1
import os 

### Function localMax  <br>
localMax performs non-maximum suppression to return a list of local maxima <br>

<b> Input: </b> <br>
     inputImg: 2D array, image to be analyzed <br>
     nLocMax: int, maximum number of local maxima to find/store <br>
     threshold: float, pixel intensity threshold for acceptance as a local maximum <br>
     minDistBtwLoMax: int, minimum distance betweeen local maxima <br>

<b> Output: </b> <br>
     maxLocList: list, tuples of (x,y) coordinates of the local maxima 


In [2]:
def localMax(inputImg, nLocMax, threshold, minDistBtwLocMax):
    
    scratch = inputImg.copy()
    nFoundLocMax = 0
    maxLocList = []
    
    for i in range(nLocMax):
        minVal, maxVal, minLoc, maxLoc = cv.minMaxLoc(scratch, None) # find the global maximum
        if (maxVal > threshold): # if the pixel intensity exceeds the specified threshold
            # store the point
            nFoundLocMax = nFoundLocMax + 1
            row = maxLoc[1]
            col = maxLoc[0]
            maxLocList.append(maxLoc)
            
            # define circular region where no other local maxima can exist
            r0 = row-minDistBtwLocMax
            if (r0<0):
                r0 = 0
                
            r1 = row+minDistBtwLocMax
            if (r1>inputImg.shape[0]):
                r1 = inputImg.shape[0]
            
            c0 = col-minDistBtwLocMax
            if (c0<0):
                c0 = 0
                
            c1 = col+minDistBtwLocMax
            if (c1>inputImg.shape[1]):
                c1 = inputImg.shape[1]
            
            # zero out a circle of pixels around the located maximum
            for r in range(r0,r1):
                for c in range(c0,c1):
                    dist = np.sqrt((row-r)**2+(col-c)**2)
                    if (dist<=minDistBtwLocMax):
                        scratch[r,c]=0.0
        
    return maxLocList

### Class Template <br>
An object of this class represents one template image used in the matching algorithm <br>

<b> Fields: </b> <br>
 image_path: string, filepath for the image <br>
 threshold: float, threshold used in choosing matches <br>
 templateImg: 2D array, the template image (array of pixel values) <br>
 h, w: height and width of the image <br>
 matches: list of tuples, (x,y) coordinates of the matches 


In [3]:
class Template:
    def __init__(self, image_path, threshold):
        self.image_path = image_path
        self.threshold = threshold
        self.templateImg = cv.imread(image_path,0)
        self.h, self.w = self.templateImg.shape[:2]
        self.matches = []

## Main Code

The following code performs template matching on all of the images contained in the input directory. DBSCAN is applied to remove false positives. Output images with match locations marked are saved in the output directory. Match coordinates are saved in an Excel file.

In [ ]:
inputDirectory = '/Volumes/KINGSTON/PLML_aligned' #Directory holding input images
outputDirectory = '/Volumes/KINGSTON/PLMLAutoResults' #Directory in which to output images with matches marked
excelFile = 'AutoAlignedResultsPLML.xls' #Name of Excel output file to store match coordinates

dfResults = [] #list of tuples (df, imageFile), df: dataframe containing x,y coords imageFile: image file name

for imageFile in os.listdir(inputDirectory):

    #import images and templates
    img = cv.imread(os.path.join(inputDirectory,imageFile),0)

    templateFiles = ["Templates/TemplateP1.jpg", "Templates/TemplateP2.jpg", "Templates/TemplateP3.jpg",
                     "Templates/TemplateP4.jpg", "Templates/TemplateP5.jpg", "Templates/TemplateP6.jpg",
                     "Templates/Template01.jpg", "Templates/Template02.jpg", "Templates/Template03.jpg",
                     "Templates/Template7.tiff", "Templates/Template05.jpg", "Templates/Template06.jpg"]

    threshVals = 0.5*np.ones((len(templateFiles),1)) #threshold values for template matching algorithm

    templates = []

    #create a Template object for each template image
    for fileName in templateFiles:
        temp = Template(fileName, threshVals[templateFiles.index(fileName)])
        templates.append(temp)

    rgb = cv.cvtColor(img,cv.COLOR_GRAY2RGB)

    #perform template matching and pick out local maximum for each match
    matchLocs = []

    for t in templates:
        res = cv.matchTemplate(img,t.templateImg,cv.TM_CCOEFF_NORMED)
        t.matches = localMax(res, 75, t.threshold, 13)

        if len(t.matches) != 0:
            #append new matches
            if len(matchLocs)==0:
                toAdd = np.array(t.matches)+np.array((t.w/2,t.h/2))
                matchLocs.extend(toAdd)
            else:
                for match in t.matches:
                    matchArr = np.array(match)+np.array((t.w/2,t.h/2))
                    distance = np.linalg.norm(np.array(matchLocs)-matchArr, axis=1)

                    if distance.min()>13:
                        matchLocs.append(matchArr)            

    #mark match locations in output image
    for point in matchLocs:
        cv.drawMarker(rgb, (int(point[0]),int(point[1])), (0,0,255), cv.MARKER_CROSS,10,2)

    #remove false positives
    points = np.array(matchLocs)
    labels = sklearn.cluster.DBSCAN(eps=50, min_samples=5).fit_predict(points)    

    posLabels = [l for l in labels if l >= 0] #find all non-negative labels
    if len(posLabels)==0:
        df = pd.DataFrame([])
        dfResults.append((df,imageFile))
    else:
        mtLabel = np.bincount(posLabels).argmax() #find the most frequently occurring label

        rgb2 = cv.cvtColor(img,cv.COLOR_GRAY2RGB)
        clusterPts = []

        for i in range(len(labels)):
            if labels[i] == mtLabel:
                cv.drawMarker(rgb2, (int(points[i][0]),int(points[i][1])), (0,0,255), cv.MARKER_CROSS,10,2)
                clusterPts.append(points[i])

        #output image with marked matches
        cv.imwrite(os.path.join(outputDirectory,imageFile),rgb2)

        df = pd.DataFrame(clusterPts, columns = ['x','y'])
        dfResults.append((df,imageFile))

#write match coordinates to Excel file
with pd.ExcelWriter(excelFile) as writer:
    for (df,fileName) in dfResults:
        df.to_excel(writer, sheet_name=fileName[19:], index=False)

writer.save()